In [1]:
import json
from collections import Counter

## Count the frequency of relevant processes

In [7]:
target_processes = {"New asset inventory", "Asset disbursement to clients", "Self-service asset check-out"}

global_counter = Counter()


for i in range(1, 31):
    if i < 10:
        i = '0' + str(i)
    
    file_path = f'../scene{i}/scene{i}_vid.json'

    with open(file_path, 'r') as f:
        
        json_file = json.load(f)

    process_counter = Counter()

    for item in json_file.get('localisations', []):
        process_name = item.get('data', {}).get('process')

        if process_name and process_name in target_processes:
            process_counter[process_name] += 1

    global_counter.update(process_counter)

    #print(process_counter)

for process in target_processes:
    print(f"{process}: {global_counter[process]}")

Asset disbursement to clients: 24
Self-service asset check-out: 18
New asset inventory: 17


## Get the start and the end of each process

In [13]:
process_timestamps = []

for i in range(1, 31):
    if i < 10:
        i = '0' + str(i)
    
    file_path = f'../scene{i}/scene{i}_vid.json'

    with open(file_path, 'r') as f:
        json_file = json.load(f)

    for item in json_file.get('localisations', []):
        process_name = item.get('data', {}).get('process')

        if process_name:
            events = item.get('localisation', [])
            if events:
                start_tcin = events[0].get('tcin')
                end_tcout = events[-1].get('tcout')
                process_timestamps.append((process_name, start_tcin, end_tcout))

# print(process_timestamps)

for item in process_timestamps:
    print(item)

('Asset disbursement to clients', '08:48:32.113', '08:50:59.438')
('Asset disbursement to clients', '08:51:18.935', '08:52:47.625')
('Asset disbursement to clients', '08:52:01.824', '08:54:55.833')
('New asset inventory', '08:57:06.000', '08:58:42.325')
('New asset inventory', '08:58:50.685', '09:00:39.047')
('New asset inventory', '09:06:13.373', '09:18:44.646')
('Defective asset return for repair', '09:09:41.484', '09:12:43.132')
('Self-service asset check-out', '09:14:13.211', '09:14:53.222')
('Self-service asset check-out', '09:15:57.817', '09:17:09.673')
('New asset inventory', '09:20:11.041', '09:20:55.051')
('New asset inventory', '09:20:17.942', '09:21:32.892')
('Asset return', '09:22:11.509', '09:23:09.911')
('Asset repair', '09:24:18.350', '09:27:22.475')
('Self-service asset check-out', '09:26:10.000', '09:26:29.508')
('New asset inventory', '09:30:02.613', '09:34:48.870')
('Self-service asset check-out', '09:30:55.103', '09:31:57.974')
('Asset return', '09:32:58.639', '09:3

In [14]:
len(process_timestamps)

98

## Filter relevant processes

In [18]:
filtered_process_timestamps = [(process_name, start_tcin, end_tcout) for process_name, start_tcin, end_tcout in process_timestamps if process_name in target_processes]

for item in filtered_process_timestamps:
    print(item)

print(f'=> the number of processes: {len(filtered_process_timestamps)}')

('Asset disbursement to clients', '08:48:32.113', '08:50:59.438')
('Asset disbursement to clients', '08:51:18.935', '08:52:47.625')
('Asset disbursement to clients', '08:52:01.824', '08:54:55.833')
('New asset inventory', '08:57:06.000', '08:58:42.325')
('New asset inventory', '08:58:50.685', '09:00:39.047')
('New asset inventory', '09:06:13.373', '09:18:44.646')
('Self-service asset check-out', '09:14:13.211', '09:14:53.222')
('Self-service asset check-out', '09:15:57.817', '09:17:09.673')
('New asset inventory', '09:20:11.041', '09:20:55.051')
('New asset inventory', '09:20:17.942', '09:21:32.892')
('Self-service asset check-out', '09:26:10.000', '09:26:29.508')
('New asset inventory', '09:30:02.613', '09:34:48.870')
('Self-service asset check-out', '09:30:55.103', '09:31:57.974')
('Self-service asset check-out', '09:36:37.712', '09:36:48.712')
('New asset inventory', '09:39:50.000', '09:41:28.000')
('New asset inventory', '09:39:49.808', '09:42:47.546')
('Asset disbursement to clien

In [48]:
from datetime import datetime

process_event_mapping = []

for i in range(1, 31):
    if i < 10:
        i = '0' + str(i)
    ocel_path = f'../scene{i}/scene{i}_video_ocel.json'

    with open(ocel_path, 'r') as f:
        ocel_data = json.load(f)

    ocel_events = ocel_data.get('ocel:events', {})


    for process_name, start_time, end_time in filtered_process_timestamps:

        start_time_dt = datetime.strptime(start_time, "%H:%M:%S.%f").time()
        end_time_dt = datetime.strptime(end_time, "%H:%M:%S.%f").time()

        matching_events = []

        for event_id, event_details in ocel_events.items():
        
            event_timestamp = event_details.get('ocel:timestamp')[11:]
            
            if event_timestamp:
                event_timestamp_dt = datetime.strptime(event_timestamp, "%H:%M:%S.%f").time()

            if start_time <= event_timestamp <= end_time:
                event_name = event_id.split('_')[0]
                matching_events.append(event_name)
        
        if len(matching_events) > 0:
            process_event_mapping.append((process_name, matching_events))


for item in process_event_mapping:
    print(item)

len(process_event_mapping)

('Asset disbursement to clients', ['Enter room', 'Chat', 'Pick asset from warehouse', 'Pick asset from warehouse', 'Check asset quality', 'Check-Out asset to user', 'Handover asset to user', 'Leave room'])
('Asset disbursement to clients', ['Enter room', 'Chat', 'Pick asset from warehouse', 'Check asset quality', 'Check-Out asset to user', 'Handover asset to user', 'Leave room', 'Enter room', 'Chat'])
('Asset disbursement to clients', ['Check-Out asset to user', 'Handover asset to user', 'Leave room', 'Enter room', 'Chat', 'Pick asset from warehouse', 'Pick asset from warehouse', 'Check asset quality', 'Check-Out asset to user', 'Handover asset to user', 'Leave room'])
('New asset inventory', ['Pick asset from warehouse', 'Unpack asset', 'Test asset quality and functionality', 'Install and configure asset', 'Create system entry for asset', 'Move asset to storage'])
('New asset inventory', ['Pick asset from self service desk', 'Unpack asset', 'Test asset quality and functionality', 'Ins

59

In [53]:
asset_disbursement_to_clients = []
new_asset_inventory = []
selfservice_asset_checkout = []

for process, events in process_event_mapping:
    if process == 'Asset disbursement to clients':
        asset_disbursement_to_clients.append(events)

    elif process == 'New asset inventory':
        new_asset_inventory.append(events)

    else:
        selfservice_asset_checkout.append(events)

In [58]:
print('Asset disbursement to clients:')

for item in asset_disbursement_to_clients:
    print(item)

print(len(asset_disbursement_to_clients))

Asset disbursement to clients:
['Enter room', 'Chat', 'Pick asset from warehouse', 'Pick asset from warehouse', 'Check asset quality', 'Check-Out asset to user', 'Handover asset to user', 'Leave room']
['Enter room', 'Chat', 'Pick asset from warehouse', 'Check asset quality', 'Check-Out asset to user', 'Handover asset to user', 'Leave room', 'Enter room', 'Chat']
['Check-Out asset to user', 'Handover asset to user', 'Leave room', 'Enter room', 'Chat', 'Pick asset from warehouse', 'Pick asset from warehouse', 'Check asset quality', 'Check-Out asset to user', 'Handover asset to user', 'Leave room']
['Create system entry for asset', 'Move asset to storage', 'Enter room', 'Pick asset from warehouse', 'Check asset quality', 'Handover asset to user', 'Leave room']
['Enter room', 'Chat', 'Pick asset from warehouse', 'Check asset quality', 'Check-Out asset to user', 'Handover asset to user', 'Leave room']
['Enter room', 'Chat', 'Pick asset from warehouse', 'Check asset quality', 'Pick asset fr

In [60]:
print('New asset inventory:')

for item in new_asset_inventory:
    print(item)

print(len(new_asset_inventory))

New asset inventory:
['Pick asset from warehouse', 'Unpack asset', 'Test asset quality and functionality', 'Install and configure asset', 'Create system entry for asset', 'Move asset to storage']
['Pick asset from self service desk', 'Unpack asset', 'Test asset quality and functionality', 'Install and configure asset', 'Create system entry for asset', 'Move asset to storage']
['Pick asset from warehouse', 'Unpack asset', 'Test asset quality and functionality', 'Create system entry for asset', 'Install and configure asset', 'Install and configure asset', 'Install and configure asset', 'Label asset', 'Move asset to storage', 'Enter room', 'Describe quality issue', 'Check asset quality', 'Check-In asset for repair', 'Move asset to repair desk', 'Pick asset from warehouse', 'Check-Out asset to user', 'Handover asset to user', 'Leave room', 'Enter room', 'Pick asset from self service desk', 'Leave room', 'Enter room', 'Pick asset from self service desk', 'Leave room']
['Pick asset from self

In [59]:
print('Self-service asset check-out:')

for item in selfservice_asset_checkout:
    print(item)

print(len(selfservice_asset_checkout))

Self-service asset check-out:
['Install and configure asset', 'Enter room', 'Pick asset from self service desk', 'Leave room']
['Label asset', 'Enter room', 'Pick asset from self service desk', 'Leave room']
['Check asset quality', 'Enter room', 'Chat', 'Pick asset from self service desk', 'Leave room']
['Install and configure asset', 'Enter room', 'Pick asset from self service desk', 'Leave room']
['Enter room', 'Pick asset from self service desk', 'Leave room']
['Chat', 'Carry out repair', 'Enter room', 'Pick asset from self service desk', 'Chat', 'Leave room']
['Chat', 'Check asset quality', 'Enter room', 'Chat', 'Pick asset from self service desk', 'Leave room']
['Chat', 'Carry out repair', 'Enter room', 'Pick asset from self service desk', 'Chat', 'Leave room']
['Enter room', 'Pick asset from self service desk', 'Leave room']
['Enter room', 'Pick asset from self service desk', 'Leave room']
['Enter room', 'Pick asset from self service desk', 'Chat', 'Leave room']
['Enter room', 'P

In [69]:
for i in range(1, 31):
    if i < 10:
        i = '0' + str(i)
    file_path = f'../scene{i}/scene{i}_vid.json'

    with open(file_path, 'r') as f:
        file_data = json.load(f)

    localisations = file_data.get('localisations', [])

    process_count = len(localisations)

    processes = []

    for item in localisations:
        process = item.get('data', {}).get('process')
        processes.append(process)

    print(f'{i}: {process_count}\n{processes}')

01: 3
['Asset disbursement to clients', 'Asset disbursement to clients', 'Asset disbursement to clients']
02: 2
['New asset inventory', 'New asset inventory']
03: 4
['New asset inventory', 'Defective asset return for repair', 'Self-service asset check-out', 'Self-service asset check-out']
04: 3
['New asset inventory', 'New asset inventory', 'Asset return']
05: 2
['Asset repair', 'Self-service asset check-out']
06: 4
['New asset inventory', 'Self-service asset check-out', 'Asset return', 'Self-service asset check-out']
07: 3
['New asset inventory', 'New asset inventory', 'Asset disbursement to clients']
08: 3
['Defective asset return for repair', 'Defective asset return for repair', 'Asset return']
09: 3
['Asset disbursement to clients', 'Asset disbursement to clients', 'Asset disbursement to clients']
10: 3
['New asset inventory', 'New asset inventory', 'Asset repair']
11: 4
['Defective asset return for repair', 'Asset repair', 'Self-service asset check-out', 'Self-service asset check-

In [72]:
ocel_path = '../global/itam_ocel.jsonl'

with open(ocel_path, 'r') as f:
    ocel_data = json.load(f)

ocel_events = ocel_data.get('ocel:events', {})
itam_process_event_mapping = []

for process_name, start_time, end_time in filtered_process_timestamps:

    start_time_dt = datetime.strptime(start_time, "%H:%M:%S.%f").time()
    end_time_dt = datetime.strptime(end_time, "%H:%M:%S.%f").time()

    matching_events = []

    for event_id, event_details in ocel_events.items():
    
        event_timestamp = event_details.get('ocel:timestamp')[11:-1]
        
        if event_timestamp:
            event_timestamp_dt = datetime.strptime(event_timestamp, "%H:%M:%S").time()

        if start_time <= event_timestamp <= end_time:
            activity = event_details.get('ocel:activity')
            matching_events.append(activity)
    
    if len(matching_events) > 0:
        itam_process_event_mapping.append((process_name, matching_events))


for item in itam_process_event_mapping:
    print(item)

len(itam_process_event_mapping)

('Asset disbursement to clients', ['checkout', 'checkout'])
('Asset disbursement to clients', ['accepted', 'accepted', 'checkout'])
('Asset disbursement to clients', ['checkout', 'accepted', 'checkout', 'checkout'])
('New asset inventory', ['create'])
('New asset inventory', ['update'])
('New asset inventory', ['create', 'update', 'checkin from', 'checkout', 'checkout', 'accepted', 'accepted', 'checkout', 'accepted', 'update'])
('Self-service asset check-out', ['accepted'])
('Self-service asset check-out', ['checkout'])
('New asset inventory', ['create'])
('New asset inventory', ['update'])
('New asset inventory', ['update'])
('Asset disbursement to clients', ['update'])
('Asset disbursement to clients', ['checkout'])
('Asset disbursement to clients', ['accepted', 'checkout', 'checkout', 'checkout'])
('Asset disbursement to clients', ['accepted', 'checkout', 'checkout', 'checkout', 'accepted', 'accepted', 'accepted', 'checkout'])
('New asset inventory', ['update'])
('New asset inventor

44